In [1]:
!pip install transformers accelerate torch pillow

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights
from torchvision import transforms
from PIL import Image
import os

print(torch.cuda.is_available())

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.8/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/local/lib/python3.8/dist-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The to

True


In [3]:
MODEL_ID = "Salesforce/blip2-opt-2.7b"
SAVE_DIR = "./blip2-opt-2.7b" 

if os.path.isdir(SAVE_DIR):
    blip2_processor = Blip2Processor.from_pretrained(SAVE_DIR, local_files_only=True)
    blip2_model = Blip2ForConditionalGeneration.from_pretrained(
        SAVE_DIR,
        torch_dtype=torch.float16,
        device_map="auto",
        local_files_only=True,
    )
else:
    # First run: download from Hub
    blip2_processor = Blip2Processor.from_pretrained(MODEL_ID)
    blip2_model = Blip2ForConditionalGeneration.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    # Save for future runs
    blip2_processor.save_pretrained(SAVE_DIR)
    blip2_model.save_pretrained(SAVE_DIR)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.42it/s]


In [4]:
for name, module in blip2_model.named_children():
    print(name, ":", module._get_name())
    # Get public methods
    public_methods = [m for m in dir(module) if callable(getattr(module, m)) and m.__contains__("forward") ]
    print("  Public methods:", public_methods)

/usr/local/lib/python3.8/dist-packages/transformers/modeling_utils.py:5006: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


vision_model : Blip2VisionModel
  Public methods: ['_hook_rss_memory_post_forward', '_hook_rss_memory_pre_forward', '_slow_forward', 'forward', 'register_forward_hook', 'register_forward_pre_hook']
qformer : Blip2QFormerModel
  Public methods: ['_hook_rss_memory_post_forward', '_hook_rss_memory_pre_forward', '_slow_forward', 'forward', 'register_forward_hook', 'register_forward_pre_hook']
language_projection : Linear
  Public methods: ['_slow_forward', 'forward', 'register_forward_hook', 'register_forward_pre_hook']
language_model : OPTForCausalLM
  Public methods: ['_hook_rss_memory_post_forward', '_hook_rss_memory_pre_forward', '_slow_forward', 'forward', 'register_forward_hook', 'register_forward_pre_hook']


In [5]:
print(blip2_model)

Blip2ForConditionalGeneration(
  (vision_model): Blip2VisionModel(
    (embeddings): Blip2VisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): Blip2Encoder(
      (layers): ModuleList(
        (0-38): 39 x Blip2EncoderLayer(
          (self_attn): Blip2Attention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (mlp): Blip2MLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((

In [8]:
teacher = blip2_model.vision_model
img = torch.randn((2,3,224,224),device='cuda')
print(teacher(img))
teacher.eval().requires_grad_(False)
teacher_dim = teacher.config.hidden_size
print("Teacher dimension:", teacher_dim)
print(f"Teacher total parameters: {sum(p.numel() for p in teacher.parameters())}")

print(teacher.config)


BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 0.0057, -0.0701, -0.1501,  ..., -0.4714, -0.8687,  0.2786],
         [-0.0491, -0.0917,  0.1030,  ...,  0.3206, -0.1873, -0.9541],
         [-0.0351,  1.1143, -0.3308,  ...,  0.0599, -1.0059, -0.4644],
         ...,
         [ 0.0833, -0.2878, -0.0047,  ...,  0.3586,  0.2903,  0.1124],
         [ 0.3037,  0.7974, -0.8682,  ...,  0.4258, -0.2030, -1.2402],
         [ 0.4199,  0.0694,  0.2830,  ...,  0.2053, -0.1896,  0.5386]],

        [[ 0.1459, -0.0856, -0.0608,  ..., -0.3530, -0.8271,  0.3594],
         [-0.0386,  0.8530, -0.0566,  ...,  0.0690, -0.4939, -0.1065],
         [ 0.4946,  0.3811,  1.4014,  ..., -0.3660, -1.4658,  0.5591],
         ...,
         [ 0.5913,  0.9463, -0.3752,  ...,  0.6357, -0.1910, -1.0908],
         [ 0.4905,  1.2158,  0.1620,  ...,  0.3718, -0.3167, -0.3457],
         [ 0.4304,  0.8438, -0.1110,  ...,  0.0414, -0.8647,  0.0812]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<NativeLayerNormBa

In [ ]:
class StudentMobileNet(nn.Module):
    def __init__(self, teacher_dim):
        super(StudentMobileNet, self).__init__()
        self.mobilenet = mobilenet_v2(weights=None) 
        self.backbone = self.mobilenet.features       
        
        with torch.no_grad():
            inp = torch.randn(1, 3, 224, 224)
            out = self.backbone(inp)
        
        self.proj = nn.Conv2d(out.shape[1], teacher_dim, kernel_size=1, stride=1, padding=0)   #(B, 1408 , H/32, W/32) -> B, 1408, 7, 7

        self.fc = nn.Linear(teacher_dim, teacher_dim)

    def forward(self, x , size_hw = (16, 16)):
        x = self.backbone(x)
        x = self.proj(x)  # Project to teacher dimension
        x = F.interpolate(x, size=size_hw, mode="bilinear", align_corners=False) # B, teacher_dim, G*G
        x = x.flatten(2).permute(0, 2, 1)  # B, G*G, teacher_dim

        # CLS made from global mean of patch tokens (then a small head)
        cls = self.fc(x.mean(dim=1, keepdim=True))  # [B, 1, D]
        tokens = torch.cat([cls, x], dim=1)               # [B, 1+G*G, D]
        return tokens



In [23]:
from torch.utils.data import Dataset,DataLoader
class CLIPDataset(Dataset):
    def __init__(self):
        super().__init__()
        self.image_dir = "../datasets/flickr8k/Images"
        self.transform = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
        ])
        self.images_dict = self.read_images(self.image_dir)

    def __len__(self):
        return len(self.images_dict)
    
    def __getitem__(self, idx):
        image_fname = list(self.images_dict.keys())[idx]
        img = self.images_dict[image_fname]
        img = self.transform(img)
        return img
    
    def read_images(self, image_dir):
        images_dict = {}
        for fname in os.listdir(image_dir):
            if fname.lower().endswith('.jpg') or fname.lower().endswith('.png'):
                image = Image.open(os.path.join(image_dir, fname)).convert('RGB')
                images_dict[fname] = image
        return images_dict



train_dataloader = DataLoader(CLIPDataset(),16,True)



In [24]:
mobilenet_v2_ = StudentMobileNet(teacher_dim)
mobilenet_v2_ = mobilenet_v2_.to('cuda')


In [25]:
def KdLoss(s, t, temp=2):
    #t -> (B,Q,D)
    #s -> (B,Q,D)
    B,Q,D = t.shape

    t = t.reshape(-1,D)  #B*Q,D
    s = t.reshape(-1,D)  #B*Q,D

    with torch.no_grad():
        pt = (t @ t.t()) / temp     # B,B
        pt = F.softmax(pt, dim=1)   # B,B
    ps = (s @ t.t()) / temp         # B,B
    log_ps = F.log_softmax(ps, dim=1)       # B,B 
    loss = (-pt * log_ps).sum(dim=1).mean()
    return loss

In [ ]:
optimizer = torch.optim.Adam(mobilenet_v2_.parameters(),lr = 1e-5)

print(sum(p.numel() for p in mobilenet_v2_.parameters()))
epoch = 3

teacher = teacher.to('cuda')
teacher.eval()

for e in range(epoch):
    print(f"*************** epoch {e+1}/{epoch}*************")
    for i,img in enumerate(train_dataloader):
        mobilenet_v2_.train()

        img = img.to('cuda')
        with torch.no_grad():
            teacher_y = teacher(img).last_hidden_state.float()
        student_y = mobilenet_v2_(img)

        mse_loss = F.mse_loss(student_y,teacher_y)
        # kd_loss = KdLoss(student_y, teacher_y) * 10

        loss = mse_loss 

        if (i)%10==0:
            print(f"loss {loss.item()}")
        

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


7292392
*************** epoch 1/3*************
loss 0.8455662131309509
loss 0.7561948299407959
loss 0.7198495864868164
loss 0.6873339414596558
loss 0.6595894694328308
loss 0.648616373538971
loss 0.6387004852294922
loss 0.6393941044807434
loss 0.6564106345176697
loss 0.6380351781845093
loss 0.6525426506996155
loss 0.6174314618110657
loss 0.6357091069221497
loss 0.6622722148895264
loss 0.6336650848388672
loss 0.6200361251831055
loss 0.6191397905349731
loss 0.6259984970092773
loss 0.6121312975883484
loss 0.6197715997695923
loss 0.6041571497917175
loss 0.6224290132522583
loss 0.6127133369445801
loss 0.6178023815155029
loss 0.6422786712646484
loss 0.6116185188293457
loss 0.61407071352005
loss 0.6205826997756958
loss 0.6146852970123291
loss 0.6058721542358398
loss 0.6092562675476074
loss 0.6226474642753601


KeyboardInterrupt: 

In [28]:
from types import SimpleNamespace
from transformers.modeling_outputs import BaseModelOutputWithPooling

class StudentVisionAdapter(nn.Module):
    """
    Drop-in replacement for BLIP-2's vision_model.
    Produces last_hidden_state [B, 1+G*G, D] and pooler_output [B, D].
    """
    def __init__(self, student_model, teacher_dim=1408, image_size=224, patch_size=14):
        super().__init__()
        self.student = student_model
        # Minimal config attributes that BLIP-2 code looks at
        self.config = SimpleNamespace(
            hidden_size=teacher_dim,
            image_size=image_size,
            patch_size=patch_size,
        )

    def forward(self, pixel_values, **kwargs):
        # Derive token grid from inputs to stay in sync with the processor
        H = pixel_values.shape[-2]
        W = pixel_values.shape[-1]
        assert H == W, "Expected square images"
        G = H // self.config.patch_size

        tok = self.student(pixel_values, size_hw=(G, G))  # [B, 1+G*G, D]
        # Keep dtype consistent with inputs (handles fp16 runs)
        tok = tok.to(dtype=torch.float16)

        print("Student output dim:",tok.dtype)
        return BaseModelOutputWithPooling(
            last_hidden_state=tok,          # [B, 1+G*G, D]
            pooler_output=tok[:, 0, :],     # use CLS as pooled vector
            hidden_states=None, 
            attentions=None
        )

In [29]:
vision_width = blip2_model.config.qformer_config.encoder_hidden_size  # e.g., 1408
student_vision = StudentVisionAdapter(
    student_model=mobilenet_v2_,
    teacher_dim=vision_width,
    image_size=blip2_processor.image_processor.size["shortest_edge"] if "shortest_edge" in blip2_processor.image_processor.size else 224,
    patch_size=14,  # BLIP-2 vision backbones use 14; keep it consistent with G=H//14
)

blip2_model.vision_model = student_vision


In [18]:
blip2_model.vision_model = teacher

In [30]:
image = Image.open("/home/ubuntu/VLM/datasets/flickr8k/Images/36422830_55c844bc2d.jpg").convert("RGB")
prompt = "Question: Describe the image Answer:"

# Preprocess input
inputs = blip2_processor(images=image, text=prompt, return_tensors="pt").to(blip2_model.device)

# Generate output
generated_ids = blip2_model.generate(**inputs, max_new_tokens=50)

generated_text = blip2_processor.decode(generated_ids[0], skip_special_tokens=True)

print(generated_text)

Student output dim: torch.float16
 a man in a blue shirt and black pants is standing in front of a white building



In [ ]:
from torchvision import datasets, transforms

cifar_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224))
])

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=cifar_transforms)

In [ ]:
import numpy as np

def eval_model(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    iter = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to('cuda')
            outputs = model(images)
            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            print(f"Iteration [{iter}/{len(data_loader)}]", end="\r")
            iter += 1
    return all_preds, all_labels

def accuracy(all_preds, all_labels):
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    return (all_preds == all_labels).mean()